#### Determine whether each series is Stationary and also compute the Density

In [2]:
import pandas as pd
from checkStationary import ADF
import numpy as np
from dateutil import relativedelta

In [3]:
datalib = "/home/tbrownex/Mastercard/"

In [8]:
def getData():
    df = pd.read_csv(datalib+"Processed.csv")
    
    df["Date"] = pd.to_datetime(df['Month'])
    del df["Month"]
    
    cols = ["Country", "Program", "Customer", "Driver", "Date", "Amount"]
    df = df[cols]
    
    sort = ["Country", "Program", "Customer", "Driver", "Date"]
    df = df.sort_values(sort)
    df = df.set_index(["Country", "Program", "Customer", "Driver"], drop=True)
    return df

In [9]:
def writeResults(pkid):
    DELIM = "|"
    filenm = "Processed+Results.csv"
    with open(datalib+filenm, "w") as output:
        hdr = "Country" +DELIM+ "Program"+DELIM+"Customer"+DELIM+"Driver"+\
        DELIM+"Length"+DELIM+"Density"+DELIM+"Stationary"
        hdr += "\n"
        output.write(hdr)
        for key, val in pkid.items():
            key = key[0] +DELIM+ key[1] +DELIM+ key[2] +DELIM+ key[3]
            rec = str(val["length"]) +DELIM+ str(val["density"]) +DELIM+ str(val["stationary"])
            rec = key +DELIM+ rec + "\n"
            output.write(rec)

In [10]:
def processDF(df):
    pkid    = {}

    prt = False
    CI = "5%"

    for idx, val in df.groupby(level=[0, 1, 2, 3]):
        results = {}
        amounts = val["Amount"]
        results["length"] = len(amounts)
        amounts = amounts.reset_index(drop=True)
        if results["length"] > 8:
            results["stationary"] = ADF(amounts, CI, prt)
        else:
            results["stationary"] = None
        dates = val["Date"]
        dates = dates.reset_index(drop=True)
        diff = relativedelta.relativedelta(dates.iloc[-1], dates[0])
        duration = 12*diff.years + diff.months
        if duration > 0:
            results["density"] = (len(dates)-1)  / duration
        else:
            results["density"] = None
        pkid[idx] = results
    return pkid

In [ ]:
df = getData()
results = processDF(df)
writeResults(results)

/home/tbrownex/tensorflow/lib/python3.5/site-packages/statsmodels/regression/linear_model.py:1510: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid
/home/tbrownex/tensorflow/lib/python3.5/site-packages/statsmodels/regression/linear_model.py:846: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
